<a href="https://colab.research.google.com/github/afrizalsofyan/Sistem-Rekomendasi-Film/blob/main/Salinan_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import pickle
from random import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
rat = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ratings.csv')

#set nama kolom 
rat = rat[['userId', 'movieId', 'rating']]

rat

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [ ]:
film = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movies.csv')

#set kolom
film = film[['movieId', 'title']]

#set makasimum index dalam tabel film
max_film = max(film['movieId'])

film

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017)
9738,193583,No Game No Life: Zero (2017)
9739,193585,Flint (2017)
9740,193587,Bungo Stray Dogs: Dead Apple (2018)


#Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from numpy.random import RandomState

#set tabel join menjadi tabel/matrik user item
useritem = pd.pivot_table(rat, index='userId', columns='movieId', values='rating', aggfunc=np.max)

#preprocessing 1
datamatrik = useritem.fillna(0)
#preprocessing 2
datamatrik = datamatrik.round()


In [ ]:
datamatrik

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,4.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#dff = datamatrik.append(pd.Series(name=''))

In [ ]:
datamatrik.loc[611, 1]=0
datamatrik.loc[611, 2]=3

In [ ]:
dff = datamatrik.fillna(0)


In [ ]:
dff

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
609,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
%%time
#userbased
pearsoncorr = datamatrik.T.corr(method='pearson')



CPU times: user 6.95 s, sys: 744 µs, total: 6.95 s
Wall time: 6.98 s


In [ ]:
%%time
cosine_sim = cosine_similarity(datamatrik)
cs = pd.DataFrame(cosine_sim)
cs.index = datamatrik.index
cs.columns = datamatrik.T.columns

CPU times: user 187 ms, sys: 26.1 ms, total: 213 ms
Wall time: 131 ms


#Proses User-based Neighbord / Top-N

Rekomendasi menggunakan Pearson Correlation dan Cosine Similarity

In [ ]:
#Fungsi untuk menghitung nilai rating 

def rekomendasi(tabel, similaritas, user, jml_user=5):
    #inisialiasi tabel rating dan sim
    itm = pd.DataFrame({'nilairating': np.zeros(max_film),
                           'sim': np.zeros(max_film)}, 
                           index=np.arange(1, 193610))
    
    #pengurutan nilai similaritas terbesar
    sim1 = similaritas[user].sort_values(ascending=False)[1:jml_user]
  
    for sim2, sim3 in sim1.items():
        hmp = (tabel[user]==0) & (np.invert(tabel[sim2]==0))
        #proses Weighted Sum
        atas = tabel[sim2][hmp] * sim3
        itm['nilairating'][atas.index] += atas.values
        itm['sim'][atas.index] += sim3
        hasil_rek = itm['nilairating'] / abs(itm['sim'])
        #print(atas)
    #return (sim1)
    #print(np.mean(sim1))
    return (hasil_rek).sort_values(ascending=False)

In [ ]:
rekomendasi(datamatrik.T, pearsoncorr, 1).dropna()

1248    5.0
1288    5.0
3262    5.0
3265    5.0
1283    5.0
       ... 
3593    1.0
1125    1.0
3421    1.0
1389    1.0
2011    1.0
Length: 766, dtype: float64

In [ ]:
# fungsi menampilkan hasil rekomendasi

def daftar_rek(user_id, tabel, similaritas, top_n=5, pref=False, rek=False, rek_cos=False):
    
    if pref:
        user_movies = (rat[rat.userId == user_id])
        user_movies = user_movies.sort_values(by='rating', ascending=False)
        print("\nPreferensi Film:\n")
        for i in range(len(user_movies)):
            movie_id = user_movies.iloc[i]['movieId']
            title = film.loc[movie_id]['title']
            rating = user_movies.iloc[i]['rating']
            print("Title: {}\nUser Rating: {}\n"
                  .format(title, rating))

    if rek:
      rek_film = rekomendasi(tabel, similaritas, user_id)[:top_n].dropna()
      list_film = list(rek_film.items())
      list_film = list_film[:top_n]
      list_film.sort(key=lambda x: x[1], reverse=True)
      #film.set_index('movieId', inplace=True)
      print("\nHasil Rekomendasi:\n")
      for movieId, rating in list_film:
          title = film.loc[movieId]['title']
          print("Id Film: {}\nJudul: {}\nPrediksi nilai reting: {}\n"
                .format(movieId, title, rating))
            
    if rek_cos:
        rek_film = rekomendasi(tabel, similaritas, user_id)[:top_n].dropna()
        list_film = list(rek_film.items())
        list_film = list_film[:top_n]
        list_film.sort(key=lambda x: x[1], reverse=True)
       
        print("\nHasil Rekomendasi:\n")
        for movieId, rating in list_film:
            title = film.loc[movieId]['title']
            print("Id Film: {}\nJudul: {}\nPrediksi nilai reting: {}\n"
                  .format(movieId, title, rating))
    

In [ ]:
daftar_rek(user_id=1, tabel=datamatrik.T, similaritas=pearsoncorr,  rek=True)


Hasil Rekomendasi:

Id Film: 1248
Judul: Hurricane Streets (1997)
Prediksi nilai reting: 5.0

Id Film: 1288
Judul: Scream 2 (1997)
Prediksi nilai reting: 5.0

Id Film: 3262
Judul: Shadows and Fog (1991)
Prediksi nilai reting: 5.0

Id Film: 3265
Judul: All That Heaven Allows (1955)
Prediksi nilai reting: 5.0

Id Film: 1283
Judul: For Richer or Poorer (1997)
Prediksi nilai reting: 5.0



#Menghitung MAE / Nilai Eror, Precesion, Recall untuk Algoritma Pearson Correlation

In [ ]:
#Fungsi untuk menghitung evaluasi model 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import multilabel_confusion_matrix


def MAEEE(tabel, similaritas, user, jml_user):
    
    sim1 = similaritas[user].sort_values(ascending=False)[1:jml_user]
    sim0 = similaritas[user].sort_values(ascending=False)
    
    itm = pd.DataFrame({'ratings': np.zeros(max_film),
                           'sum': np.zeros(max_film)}, 
                           index=np.arange(1, 193610))
    
    
    itm2 = pd.DataFrame({'ratings': np.zeros(max_film),
                           'sum': np.zeros(max_film)}, 
                           index=np.arange(1, 193610))
    
    pjg=0
    pjg2=0
    cba1 = 0

    for sim2, sim3 in sim1.items():
        
        hmp = (tabel[user]==0) & (np.invert(tabel[sim2]==0))
        
        lg = tabel[sim2][hmp] * sim3
       
        itm['ratings'][lg.index] += lg.values
        itm['sum'][lg.index] += sim3
        
        hsl1 = itm['ratings'] / itm['sum']
        hsl_p = hsl1.dropna()
        pred=hsl1.fillna(0).round()
    
    for simx, simy in sim0.items():
        
        ll = tabel[user]
       
        itm2['ratings'] = ll
        
        hsl2 = itm2['ratings'] 
        hsl_a = hsl2.dropna()
        awal=hsl2.fillna(0)
    totaal = 0
    totaal = awal-pred
    totalbanget = np.mean(abs(totaal))
    pre = rekomendasi(tabel, similaritas,user)
    #tp=len(pre.index)
    #print(tp)
   
    print('nilai MAE : ' + str(totalbanget))
    print('nilai MAE2 : '+ str( mean_absolute_error(awal.round(), pred.round())))
    print('nilai Akurasi : '+ str(accuracy_score(awal.round(), pred.round())))
    print('nilai Precison : '+ str(np.sum( precision_score(awal, pred, average=None, zero_division=0))))
    print('nilai recall : '+ str(np.sum(recall_score(awal, pred, average=None, zero_division=0))))

In [ ]:
%%time
MAEEE(tabel=datamatrik.T, similaritas=pearsoncorr, user=1, jml_user=5)

nilai MAE : 0.01749918650475959
nilai MAE2 : 0.01749918650475959
nilai Akurasi : 0.9948452809528483
nilai Precison : 0.9987969488132833
nilai recall : 0.9960388257135027
CPU times: user 4.28 s, sys: 31.8 ms, total: 4.31 s
Wall time: 4.32 s
